In [4]:
import joblib
import torch
from model import EngineRULPredictor

import pandas as pd

In [8]:
scaler = joblib.load('models/scaler.pkl')

In [10]:

input_size = 18 # Sensors (21) + Settings (3)
model = EngineRULPredictor(input_size=input_size, hidden_size=512, num_layers=2, dropout=0.2)

model.load_state_dict(torch.load('models/lstm_model.pth', map_location=torch.device('cpu')))

model.eval()

EngineRULPredictor(
  (lstm): LSTM(18, 512, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Sequential(
    (0): Linear(in_features=512, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [11]:
print("Loading model...")
input_size = 18 # Sensors (21) + Settings (3)
model = EngineRULPredictor(input_size=input_size, hidden_size=512, num_layers=2, dropout=0.2)

model.load_state_dict(torch.load('models/lstm_model.pth', map_location=torch.device('cpu')))
model.eval()


scaler = joblib.load('models/scaler.pkl') # <--- 2. Load the scaler

col_names = ['unit_nr', 'time_cycles',
                'setting_1', 'setting_2', 'setting_3'] + [f"s_{i}" for i in range(1, 22)]
raw_df = pd.read_csv("data/test_FD001.txt", sep=r"\s+", header=None, names=col_names)
sensor_df = raw_df




Loading model...


In [13]:
unit_nr = 7  # Example unit number
max_rul = 542  # use the same value you trained with

index_names = ['unit_nr', 'time_cycles']
setting_names = ['setting_1', 'setting_2', 'setting_3']
sensor_names = ['s_{}'.format(i) for i in range(1, 22)] 
col_names = index_names + setting_names + sensor_names
df = pd.read_csv("data/test_FD001.txt", sep="\s+", header=None, names=col_names)
df = df[df.unit_nr == unit_nr]

df = df.drop(columns=["unit_nr", "time_cycles"])  # Drop non-feature columns for scaling
scaled = scaler.transform(df)
df = pd.DataFrame(scaled, columns=df.columns, index=df.index)

features_to_drop = ["s_1", "s_5", "s_10", "s_16", "s_18", "s_19"]
df = df.drop(columns=features_to_drop)

input_tensor = torch.tensor(df.to_numpy(), dtype=torch.float32)
input_tensor = input_tensor.unsqueeze(0)  # Add batch dimension


# Perform prediction
with torch.no_grad():
    prediction = model(input_tensor)
    print(f"Raw model prediction: {prediction.item()}")
    scaled_prediction = prediction.item() * max_rul
    print(f"Scaled RUL prediction for unit {unit_nr}: {scaled_prediction} cycles")



Raw model prediction: 0.08190962672233582
Scaled RUL prediction for unit 7: 44.39501768350601 cycles


<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_793294/319632188.py:8: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv("data/test_FD001.txt", sep="\s+", header=None, names=col_names)
